In [2]:
import pandas as pd
import seaborn as sns

In [3]:
df = pd.read_csv('./data/gemstone.csv')

In [5]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
df.describe()

,id,carat,depth,table,x,y,z,price
count,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000
mean,96786.000000,0.790688,61.820574,57.227675,5.715312,5.720094,3.534246,3969.155414
std,55879.856166,0.462688,1.081704,1.918844,1.109422,1.102333,0.688922,4034.374138
min,0.000000,0.200000,52.100000,49.000000,0.000000,0.000000,0.000000,326.000000
25%,48393.000000,0.400000,61.300000,56.000000,4.700000,4.710000,2.900000,951.000000
50%,96786.000000,0.700000,61.900000,57.000000,5.700000,5.720000,3.530000,2401.000000
75%,145179.000000,1.030000,62.400000,58.000000,6.510000,6.510000,4.030000,5408.000000
max,193572.000000,3.500000,71.600000,79.000000,9.650000,10.010000,31.300000,18818.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [10]:
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [12]:
df.duplicated().sum()

0

In [14]:
X = df.drop(labels=['price'], axis=1)

In [15]:
y = df['price']

In [16]:
categorical_col = X.select_dtypes(include='object').columns
neumerical_col = X.select_dtypes(exclude='object').columns

In [17]:
categorical_col

Index(['cut', 'color', 'clarity'], dtype='object')

In [18]:
neumerical_col

Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [22]:
X['clarity'].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [23]:
cut_category = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_category = ['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_category = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [24]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [28]:
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[cut_category, color_category, clarity_category])),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, neumerical_col),
    ('cat_pipeline', cat_pipeline, categorical_col)
])

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=30)

In [30]:
X_train  = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [31]:
X_train

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,1.274919,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.652673,1.655662,1.604964
1,-1.102907,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,1.687566,0.056195,0.481725
2,-0.997613,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-1.417114,-0.476960,-0.079894
3,-1.173764,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-1.417114,-0.476960,-1.203134
4,-0.914812,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.652673,-1.010116,1.604964
...,...,...,...,...,...,...,...,...,...,...
135496,-0.544112,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,0.652673,0.056195,0.481725
135497,-0.905908,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,0.652673,-0.476960,-0.641514
135498,0.034280,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,1.687566,-0.476960,0.481725
135499,-0.873277,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,0.652673,-0.476960,-0.079894


In [32]:
X_test

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.473004,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-1.417114,0.056195,-1.203134
1,-0.573005,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-0.382220,1.122506,0.481725
2,1.581092,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.652673,1.122506,1.043345
3,-0.218647,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-1.417114,-0.476960,-1.203134
4,-1.160998,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-1.417114,0.056195,-0.079894
...,...,...,...,...,...,...,...,...,...,...
58067,1.713008,1.013460,1.185958,-0.642862,1.003556,1.041924,1.151941,-0.382220,1.122506,-1.203134
58068,0.662413,-0.997058,0.260701,-1.164193,-1.141917,-1.126331,-1.108659,0.652673,-1.543271,2.166584
58069,-0.931959,-0.197174,-3.347799,1.442462,0.102096,0.071199,-0.224706,-1.417114,-1.010116,-1.203134
58070,-1.198492,-0.824110,-0.201927,-0.121531,-0.853450,-0.881382,-0.876803,0.652673,-0.476960,0.481725


In [33]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [34]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [35]:
regression.coef_

array([-1.76038520e-01,  6.54772305e+03, -2.36703831e+02, -1.79529877e+02,
       -2.12180557e+03, -5.49492317e+02, -3.49871010e+01, -1.08021321e+01,
       -1.81097599e+02,  1.20515188e+02])

In [36]:
regression.intercept_

3970.7662895476747

In [37]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)

    return mae, mse, rmse, r2_square

In [42]:
models = {
    "LinearRegression":LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet": ElasticNet(),
}
trained_model_list = []
model_list = []
r2_list = []


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_predict  = model.predict(X_test)

    mae, mse, rmse, r2_square = evaluate_model(y_test, y_predict)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Taining Performance')
    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("R2_square: ", r2_square)

    r2_list.append(r2_square)

    print("="*35)
    print("\n")

LinearRegression
Model Taining Performance
RMSE:  1198.8288094732538
MAE:  726.0530257868506
R2_square:  0.9117707544818099


Lasso
Model Taining Performance
RMSE:  1198.7116979598877
MAE:  727.4438430668454
R2_square:  0.9117879915646933


Ridge
Model Taining Performance
RMSE:  1198.8300003385175
MAE:  726.0818699021369
R2_square:  0.9117705791954053


ElasticNet
Model Taining Performance
RMSE:  1631.1190648100303
MAE:  1134.3638110131233
R2_square:  0.8366686502682896


